In [ ]:
# The code was removed by DSX for sharing.

In [556]:
# The code was removed by DSX for sharing.

In [557]:
# Pull data from JSON files

test_data_dict = get_data(credentials_3)
training_data_dict = get_data(credentials_1)
trial_data_dict = get_data(credentials_2)

In [558]:
# HELPER FUNCTIONS SECTION

#1. Function for getting max number of features
def getMaxFeatures(X_Train):
    max_feature=0
    for i in X_Train:
        for j in i:
            if j>max_feature:
                max_feature=j
    return max_feature+1
# print("Max Features: ", getMaxFeatures(data))

#2. Function for getting sentiment label for input sentiment score
# Function returns label value for the score residing in the defined range
def getSentimentLabel(score):    
    if score < -0.5:
        return -1
    if score >= -0.5 and score < 0:
        return -0.5
    if score >= 0 and score < 0.5:
        return 0.5
    if score >= 0.5:
        return 1


In [559]:
# Define empty lists for outputs
text_train=[] 
sentiment_train=[]
sentiment_output=[]

# Training data
for i in range(0,len(training_data_dict)):
    title = training_data_dict[i].get("title")
#     print(title)
    text_train.append(title.encode('utf-8'))
    sentiment_train.append(getSentimentLabel(training_data_dict[i].get("sentiment")))
#     print(training_data_dict[i].get("sentiment"),"Converted->",getSentimentLabel(training_data_dict[i].get("sentiment")))
    sentiment_output.append(training_data_dict[i].get("sentiment"))
    training_data_dict[i]["title"] = title.encode('utf-8')


# Trial data
text_trial=[] 
sentiment_trial=[]
for i in range(0,len(trial_data_dict)):
    title_1 = trial_data_dict[i].get("title")
    
    text_trial.append(title_1.encode('utf-8'))
    text_train.append(title_1.encode('utf-8'))
    sentiment_trial.append(getSentimentLabel(trial_data_dict[i].get("sentiment")))
#     print(trial_data_dict[i].get("sentiment"),"Converted->",getSentimentLabel(trial_data_dict[i].get("sentiment")))
    sentiment_train.append(getSentimentLabel(trial_data_dict[i].get("sentiment")))    
    sentiment_output.append(trial_data_dict[i].get("sentiment"))
    trial_data_dict[i]["title"] = title_1.encode('utf-8')

# text_train = [s.encode('utf-8') for s in text_train]
# text_trial = [s.encode('utf-8') for s in text_trial]


# Test data
test_data=[]
for i in range(0,len(test_data_dict)):
    title_2 = test_data_dict[i].get("title")
    test_data.append(title_2.encode('utf-8'))
    test_data_dict[i]["title"] = title_2.encode('utf-8')

# test_data = [s.encode('utf-8') for s in test_data]    
    
print("Training length: ",len(training_data_dict),", trial length: ",len(trial_data_dict),", TEST length: ",len(test_data_dict))
print(len(text_train), len(sentiment_train), len(sentiment_output), len(test_data))

('Training length: ', 1142, ', trial length: ', 14, ', TEST length: ', 491)
(1156, 1156, 1156, 491)


In [560]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# tokenizer = Tokenizer(num_words=1000)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

# print(text_train[0])
sequences = tokenizer.texts_to_sequences(text_train)
# print(sequences[0])

# data = pad_sequences(sequences, maxlen=50)
data = pad_sequences(sequences)
print("X_Data shape: ", data.shape)

('X_Data shape: ', (1156, 18))


In [326]:
# NOT WORKING WITH SKLEARN COUNT VERSTORIZER

#from sklearn.feature_extraction import DictVectorizer
#vec = DictVectorizer()
#vec.fit_transform(training_data_dict[0].get("title")).toarray()
#vec.get_feature_names()

# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer(tokenizer=tokenize)
# X_train_counts = count_vect.fit_transform(training_data_dict[0].get("title")).toarray()
# X_train_counts.shape

# s=training_data_dict[0].get("title")
# from sklearn.feature_extraction.text import CountVectorizer
# vec = CountVectorizer()
# data = vec.fit_transform(list(s))
# print(data)

# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer(stop_words=None)
# X_Train = count_vect.fit_transform(text_train).toarray()
# X_Eval= count_vect.fit_transform(text_trial).toarray()
# print(X_Train.shape)
# print(1 in X_Train[1])
# print(l)

In [561]:
import numpy as np

data_Y = np.array(sentiment_train)
output_Y = np.array(sentiment_output)
print(data_Y.shape)
print(output_Y.shape)

(1156,)
(1156,)


In [562]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D


# Constants
MAX_FEATURES = getMaxFeatures(data)
NO_OF_EPOCHS = 100
input_len = data.shape[1]
val_split = 0.2
batch = 50

In [329]:
# Exp_1

print('Build model...')

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(MAX_FEATURES, 50, input_length=input_len))
#model.add(Flatten())
model.add(Dropout(0.5))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(512, 3, strides=5, activation="sigmoid"))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(4, activation="sigmoid"))
model.add(Dropout(0.2))
# model.add(Activation('sigmoid'))

model.add(Dense(1, activation="tanh"))
# model.add(Activation('sigmoid'))

model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

model.fit(data, data_Y,
          batch_size=batch,
          validation_split=val_split,
          epochs=NO_OF_EPOCHS)

# Test Similarity - 0.84507313


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_87 (Embedding)     (None, 18, 50)            138750    
_________________________________________________________________
dropout_125 (Dropout)        (None, 18, 50)            0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 4, 512)            77312     
_________________________________________________________________
global_max_pooling1d_36 (Glo (None, 512)               0         
_________________________________________________________________
dense_140 (Dense)            (None, 4)                 2052      
_________________________________________________________________
dropout_126 (Dropout)        (None, 4)                 0         
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 5     

In [499]:
# Exp_2

model = Sequential()
model.add(Embedding(MAX_FEATURES, 16, input_length=input_len))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(1, activation='tanh'))
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

model.fit(data, data_Y, validation_split=val_split, epochs=NO_OF_EPOCHS,verbose = 1)

# Test Similarity - 0.83194101


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_101 (Embedding)    (None, 18, 16)            44368     
_________________________________________________________________
lstm_58 (LSTM)               (None, 50)                13400     
_________________________________________________________________
dense_163 (Dense)            (None, 1)                 51        
Total params: 57,819
Trainable params: 57,819
Non-trainable params: 0
_________________________________________________________________
Train on 913 samples, validate on 229 samples
Epoch 1/100
913/913 [==============================] - 6s - loss: 0.3938 - acc: 0.0000e+00 - val_loss: 0.3764 - val_acc: 0.0000e+00
Epoch 2/100
913/913 [==============================] - 0s - loss: 0.3784 - acc: 0.0000e+00 - val_loss: 0.3584 - val_acc: 0.0000e+00
Epoch 3/100
913/913 [==============================] - 0s - loss: 0.3330 - acc: 0.0011 - val

In [563]:
# New Exp 1

model = Sequential()
model.add(Embedding(MAX_FEATURES, 50, input_length=input_len))
model.add(Dropout(0.5))
model.add(Conv1D(512,3,padding='valid',activation='sigmoid', strides=1))
model.add(MaxPooling1D(pool_size=16))
model.add(LSTM(50))
# model.add(Dropout(0.2))
model.add(Dense(4, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='tanh'))

model.compile(loss='mse',
             optimizer='rmsprop',
             metrics=['accuracy'])
print('Train...')

model.summary()
model.fit(data, data_Y,
         epochs=NO_OF_EPOCHS,validation_split=val_split, batch_size=batch)

# Test 1 Similarity --> 0.80637867 EPOCH - 100, filters - 512, kernels - 3
# Test 2 Similarity --> 0.83535004 Epoch - 150, filters - 512, kernels - 3
# Test 3 Similarity --> 0.84114928 Epoch - 100, filters - 300, kernels - 2
# Test 4 Similarity --> 0.8526335 Epoch - 100, filters - 512, kernels - 2


Train...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_105 (Embedding)    (None, 18, 50)            138750    
_________________________________________________________________
dropout_152 (Dropout)        (None, 18, 50)            0         
_________________________________________________________________
conv1d_77 (Conv1D)           (None, 16, 512)           77312     
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 1, 512)            0         
_________________________________________________________________
lstm_62 (LSTM)               (None, 50)                112600    
_________________________________________________________________
dense_170 (Dense)            (None, 4)                 204       
_________________________________________________________________
dropout_153 (Dropout)        (None, 4)                 0         
_

In [564]:
from sklearn.metrics.pairwise import cosine_similarity

predictions = model.predict(data)

predictedScores = [pred[0] for pred in predictions]
# print(predictions)

predictedScores_reshaped = np.array(predictedScores).reshape(1,-1)
print("Predicted new shape: ",predictedScores_reshaped.shape)

# print("Output shape: ", output_Y.shape)
output_Y_reshaped = np.array(output_Y).reshape(1,-1)
print("Output new shape: ", output_Y_reshaped.shape)


similarity = cosine_similarity(predictedScores_reshaped, output_Y_reshaped)
print("Similarity: ", similarity)

('Predicted new shape: ', (1, 1156))
('Output new shape: ', (1, 1156))
('Similarity: ', array([[ 0.84974097]]))


In [ ]:
pred_norm = np.linalg.norm(predictedScores_reshaped)
orig_norm = np.linalg.norm(output_Y_reshaped)

cos_weight = pred_norm/orig_norm

final_cos = cos_weight * similarity[0][0]
print(final_cos)

In [566]:
df = pd.DataFrame.from_dict(training_data_dict+trial_data_dict)
df['Predicted Sentiment Score'] = predictedScores

df.head()

,company,id,sentiment,title,Predicted Sentiment Score
0,Morrisons,2,0.430,Morrisons book second consecutive quarter of s...,0.464958
1,IMI,3,-0.344,IMI posts drop in first-quarter organic revenu...,-0.474674
2,Glencore,4,0.340,Glencore to refinance its short-term debt earl...,0.478118
3,Ryanair,5,0.259,EasyJet attracts more passengers in June but s...,0.479872
4,Barclays,6,-0.231,Barclays 'bad bank' chief to step down,-0.472625


In [567]:
for i in range(len(predictions)):
    print(training_data_dict[i]['title'],training_data_dict[i]['sentiment'], "%.3f" % predictions[i])

('Morrisons book second consecutive quarter of sales growth', 0.43, '0.465')
('IMI posts drop in first-quarter organic revenue; warns on full year', -0.344, '-0.475')
('Glencore to refinance its short-term debt early, shares rise', 0.34, '0.478')
('EasyJet attracts more passengers in June but still lags Ryanair', 0.259, '0.480')
("Barclays 'bad bank' chief to step down", -0.231, '-0.473')
('Bilfinger Industrial Services win \xc3\x82\xc2\xa3100m BP contract extension', 0.113, '0.474')
('Bilfinger Industrial Services win \xc3\x82\xc2\xa3100m BP contract extension', 0.424, '0.474')
('Barclays share price subdued as bank faces fresh forex probe', -0.373, '-0.497')
('Centrica prepared for takeover approach - chairman', 0.111, '0.483')
("Jim Armitage: Spare no tears as Glencore's bosses are paying such a small price", -0.231, '-0.425')
('Barclays, Deutsche Bank Fight to Lift Profit Just Got Harder', -0.221, '-0.528')
('Kingfisher set to open another 200 Screwfix stores', 0.541, '0.909')
('Di

IndexError: list index out of range

In [568]:
# training_data_dict
df = pd.DataFrame.from_dict(training_data_dict+trial_data_dict)
df['Predicted Sentiment Score'] = predictedScores
df=({
   "ID":df["id"],
   "COMPANY":df["company"],
   "TITLE":df["title"],
   "SENTIMENT":df["sentiment"],
"Predicted Sentiment Score":df['Predicted Sentiment Score']
})

df = pd.DataFrame.from_dict(df)
if os.path.exists("Task2_Train.csv"):
    os.remove("Task2_Train.csv")

df.to_csv('Task2_Train.csv', index = False)

df

,COMPANY,ID,Predicted Sentiment Score,SENTIMENT,TITLE
0,Morrisons,2,0.464958,0.430,Morrisons book second consecutive quarter of s...
1,IMI,3,-0.474674,-0.344,IMI posts drop in first-quarter organic revenu...
2,Glencore,4,0.478118,0.340,Glencore to refinance its short-term debt earl...
3,Ryanair,5,0.479872,0.259,EasyJet attracts more passengers in June but s...
4,Barclays,6,-0.472625,-0.231,Barclays 'bad bank' chief to step down
5,BP,7,0.474381,0.113,Bilfinger Industrial Services win Â£100m BP co...
6,Bilfinger Industrial Services,8,0.474381,0.424,Bilfinger Industrial Services win Â£100m BP co...
7,Barclays,9,-0.496570,-0.373,Barclays share price subdued as bank faces fre...
8,Centrica PLC,10,0.482895,0.111,Centrica prepared for takeover approach - chai...
9,Glencore,11,-0.425364,-0.231,Jim Armitage: Spare no tears as Glencore's bos...


In [456]:
# Ignore if installed

!pip install --user --upgrade pixiedust

    100% |████████████████████████████████| 174kB 5.1MB/s eta 0:00:01
    100% |████████████████████████████████| 798kB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 5.6MB 202kB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 8.5MB/s eta 0:00:01
Requirement already up-to-date: astunparse in /usr/local/src/bluemix_jupyter_bundle.v69/notebook/lib/python2.7/site-packages (from pixiedust)
    100% |████████████████████████████████| 81kB 8.1MB/s eta 0:00:01
    100% |████████████████████████████████| 491kB 2.3MB/s eta 0:00:01
Requirement already up-to-date: jupyter-client>=4.2.0 in /usr/local/src/bluemix_jupyter_bundle.v69/notebook/lib/python2.7/site-packages (from jupyter_kernel_gateway->pixiedust)
Requirement already up-to-date: requests<3.0,>=2.7 in /usr/local/src/bluemix_jupyter_bundle.v69/notebook/lib/python2.7/site-packages (from jupyter_kernel_gateway->pixiedust)
    100% |████████████████████████████████| 133kB 5.7MB/s eta 0:00:01
    100% |█████████

In [ ]:
# NOT WORKING

# import pixiedust
# pixiedust.enableJobMonitor()
# display(df)

In [569]:
# Test Data Evaluation

sequences = tokenizer.texts_to_sequences(test_data)
data = pad_sequences(sequences)
predictions_TEST = model.predict(data)

for i in range(len(predictions_TEST)):
    print(test_data_dict[i]['title'], "%.3f" % predictions_TEST[i])

('Ashtead to buy back shares, full-year profit beats estimates', '0.716')
("EU regulators clear Shell's takeover of BG Group", '0.110')
("UK's FTSE has worst day so far in 2015 as BG and Prudential fall", '-0.524')
('GlaxoSmithKline acquires HIV assets', '0.908')
('Barclays faces another heavy forex fine', '-0.672')
('Diageo Shares Surge on Report of Possible Takeover by Lemann', '0.492')
('Borealis Infrastructure putting together new Severn Trent bid', '0.480')
('FTSE 100 falls as China devaluation hits Burberry, mining stocks', '-0.468')
('London Stock Exchange \xc3\xa2\xe2\x82\xac\xe2\x80\x9c Deutsche Boerse merger under threat from Brexit', '-0.327')
('Tesco Abandons Video-Streaming Ambitions in Blinkbox Sale', '0.249')
('UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to $300 mln', '0.466')
('Homebuilders, Shire Gains Spur FTSE 100 to Rally Beyond Record', '0.905')
('CompaniesRoyal Mail adds a penny to stamp prices', '-0.372')
('Royal Mail shares fall as Ofcom publish

In [570]:
# test_data_dict
df = pd.DataFrame.from_dict(test_data_dict)
df['Predicted Sentiment Score'] = predictions_TEST
df=({
   "ID":df["id"],
   "COMPANY":df["company"],
   "TITLE":df["title"],
"Predicted Sentiment Score":df['Predicted Sentiment Score']
})

df = pd.DataFrame.from_dict(df)
if os.path.exists("Task2_Test.csv"):
    os.remove("Task2_Test.csv")

df.to_csv('Task2_Test.csv', index = False)

df

,COMPANY,ID,Predicted Sentiment Score,TITLE
0,Ashtead,1144,0.715844,"Ashtead to buy back shares, full-year profit b..."
1,Shell,1145,0.110463,EU regulators clear Shell's takeover of BG Group
2,Prudential,1146,-0.524386,UK's FTSE has worst day so far in 2015 as BG a...
3,GlaxoSmithKline,1147,0.908108,GlaxoSmithKline acquires HIV assets
4,Barclays,1148,-0.672149,Barclays faces another heavy forex fine
5,Diageo,1149,0.492314,Diageo Shares Surge on Report of Possible Take...
6,Borealis Infrastructure,1150,0.480204,Borealis Infrastructure putting together new S...
7,Burberry Group plc,1151,-0.468063,FTSE 100 falls as China devaluation hits Burbe...
8,Deutsche Boerse,1152,-0.327209,London Stock Exchange â€“ Deutsche Boerse merg...
9,Tesco PLC,1153,0.249337,Tesco Abandons Video-Streaming Ambitions in Bl...
